# VISUALIZE_TRAJECTORIES
This notebook creates and saves mp4 videos for the model policies.

It is the only visualize notebook that does not rely on pre-generated trajectories.

It would probably be faster if we just visualized the already generated trajectories, but alas, we have not done that yet. 

This file asks for you to give it the location of the models, and then - through aptly named functions I might add - visualizes the trajectories in an animated video as it simulates for them.

We can control the scene generated through the use of a random seed, so that all models are in a fair comparison.

Displaying all of the models on the same frame would be hectic, so instead we limit to displaying 2 at a time. We do model[i] and model[i+1] at the same time, so when defining model_labels keep that in mind or change it how you see fit.

In [1]:
%matplotlib inline

import h5py
from IPython.display import HTML
from matplotlib import animation
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import ndimage
import sys
import tensorflow as tf

import hgail.misc.utils

import hyperparams
import utils
import validate

/home/dphillips/miniconda3/envs/rllab3/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
basedir = '../../data/experiments/'
model_labels = [
    'multiagent_rails_col_off_5k_2k_1_fine',
    'multiagent_curr_1_fine',
    'multiagent_rails_col_off_5k_2k_2_fine',
    'multiagent_curr_2_fine',
    'multiagent_rails_col_off_5k_2k_3_fine',
    'multiagent_curr_3_fine'
]
itrs = [
    200,
    200,
    200,
    200,
    200,
    200
]
model_params_filepaths = [os.path.join(basedir, label, 'imitate/log/itr_' + str(itrs[i]) + '.npz') 
                          for i,label in enumerate(model_labels)]
model_args_filepaths = [os.path.join(basedir, label, 'imitate/log/args.npz') for label in model_labels]
n_models = len(model_labels)

multi = True

In [ ]:
print(model_params_filepaths)

In [3]:
def mutliagent_simulate(env, policy, max_steps, env_kwargs=dict(), render_kwargs=dict()):
    x = env.reset(**env_kwargs)
    n_agents = x.shape[0]
    traj = hgail.misc.simulation.Trajectory()
    dones = [True] * n_agents
    policy.reset(dones)
    imgs = []
    for step in range(max_steps):
        sys.stdout.write('\rstep: {} / {}'.format(step+1, max_steps))
        img = env.render(**render_kwargs)
        imgs.append(img)
        a, a_info = policy.get_actions(x)
        nx, r, dones, e_info = env.step(a)
        traj.add(x, a, r, a_info, e_info)
        if any(dones): break
        x = nx
    return imgs

In [4]:
def create_render_map(model_labels, model_args_filepaths, model_params_filepaths, 
                      multi=False, rand=None, max_steps=200, n_vehs=None, remove_ngsim=False):
    render_map = dict()
    env_kwargs = dict()
    if rand != None:
        env_kwargs = dict(random_seed=rand)
    if not multi:
        env_kwargs = dict(
            egoid=worst_egoid, 
            start=worst_start
        )
    render_kwargs = dict(
        camera_rotation=45.,
        canvas_height=500,
        canvas_width=600
    )
    for i in range(len(model_labels)):
        print('\nrunning: {}'.format(model_labels[i]))

        # create session
        tf.reset_default_graph()
        sess = tf.InteractiveSession()

        # load args and params
        args = hyperparams.load_args(model_args_filepaths[i])

        print('\nargs loaded from {}'.format(model_args_filepaths[i]))
        if multi:
            args.env_multiagent = True
            if remove_ngsim:
                args.remove_ngsim_veh = True

            if n_vehs:
                args.n_envs = 100
                args.n_vehs = 100
        params = hgail.misc.utils.load_params(model_params_filepaths[i])
        print('\nparams loaded from {}'.format(model_params_filepaths[i]))

        # load env and params
        env, _, _ = utils.build_ngsim_env(args)

        normalized_env = hgail.misc.utils.extract_normalizing_env(env)
        if normalized_env is not None:
            normalized_env._obs_mean = params['normalzing']['obs_mean']
            normalized_env._obs_var = params['normalzing']['obs_var']

        # load policy
        if 'hgail' in model_labels[i]:
            policy = utils.build_hierarchy(args, env)
        else:
            policy = utils.build_policy(args, env)

        # initialize variables
        sess.run(tf.global_variables_initializer())

        # load params
        if 'hgail' in model_labels[i]:
            for j, level in enumerate(policy):
                level.algo.policy.set_param_values(params[j]['policy'])
            policy = policy[0].algo.policy
        else:
            policy.set_param_values(params['policy'])

        # collect imgs
        if args.env_multiagent:
            imgs = mutliagent_simulate(
                env, 
                policy, 
                max_steps=max_steps, 
                env_kwargs=env_kwargs,
                render_kwargs=render_kwargs
            )
        else:
            imgs = simulate(
                env, 
                policy, 
                max_steps=max_steps, 
                env_kwargs=env_kwargs,
                render_kwargs=render_kwargs
            )
        render_map[model_labels[i]] = imgs
    return render_map

In [5]:
def do_it_all_once(model_labels, model_args_filepaths, model_params_filepaths,
                   multi=False, name='single_multi', single_multi_comp=1, rand=None, n_vehs=None,
                  remove_ngsim=False):
    #do this with just 2 models at a time.
    print("creating render map for: ", "; ".join(model_labels))
    render_map = create_render_map(model_labels, model_args_filepaths, model_params_filepaths, multi, 
                                   rand, n_vehs=n_vehs, remove_ngsim=remove_ngsim)
    imgs = [np.concatenate((a,b), 0) for (a,b) in zip(*[render_map[i] for i in model_labels])]
    fig, ax = plt.subplots(figsize=(16,16))
    plt.title(name)
    print("\nplotting")
    
    img = plt.imshow(imgs[0])

    def animate(i):
        img.set_data(imgs[i])
        return (img,)

    anim = animation.FuncAnimation(
        fig, 
        animate, 
        frames=len(imgs), 
        interval=100, 
        blit=True
    )

    WriterClass = animation.writers['ffmpeg']
    writer = WriterClass(fps=10, metadata=dict(artist='bww'), bitrate=1800)
    anim.save('../../data/media/' + name + '.mp4', writer=writer)
    print("Saved: ", name)

    HTML(anim.to_html5_video())
    plt.close()

In [6]:
remove_ngsim_vehicles = False
for i in range(0,5):
    print("\Run number: ", i)
    seed = i
    for j in [1,2,3]: #number of models to 'compare'
        indx = (j-1)*2
        name = "-".join(model_labels[indx:indx+2])+'_'+str(i)+"_"+str(seed)
        if remove_ngsim_vehicles:
            name = name + '_ngsim_removed'
        do_it_all_once(model_labels[indx:indx+2], 
                       model_args_filepaths[indx:indx+2], 
                       model_params_filepaths[indx:indx+2], 
                       multi=True, 
                       name=name, 
                       single_multi_comp=j, 
                       rand=seed,
                       n_vehs=100,
                       remove_ngsim=remove_ngsim_vehicles)
        print("\nDone once.\n")


    

\Run number:  0
creating render map for:  multiagent_rails_col_off_5k_2k_1_fine; multiagent_curr_1_fine

running: multiagent_rails_col_off_5k_2k_1_fine

args loaded from ../../data/experiments/multiagent_rails_col_off_5k_2k_1_fine/imitate/log/args.npz

params loaded from ../../data/experiments/multiagent_rails_col_off_5k_2k_1_fine/imitate/log/itr_200.npz


/home/dphillips/miniconda3/envs/rllab3/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


step: 200 / 200
running: multiagent_curr_1_fine

args loaded from ../../data/experiments/multiagent_curr_1_fine/imitate/log/args.npz

params loaded from ../../data/experiments/multiagent_curr_1_fine/imitate/log/itr_200.npz
step: 200 / 200
plotting
Saved:  multiagent_rails_col_off_5k_2k_1_fine-multiagent_curr_1_fine_0_0

Done once.

creating render map for:  multiagent_rails_col_off_5k_2k_2_fine; multiagent_curr_2_fine

running: multiagent_rails_col_off_5k_2k_2_fine

args loaded from ../../data/experiments/multiagent_rails_col_off_5k_2k_2_fine/imitate/log/args.npz

params loaded from ../../data/experiments/multiagent_rails_col_off_5k_2k_2_fine/imitate/log/itr_200.npz
step: 200 / 200
running: multiagent_curr_2_fine

args loaded from ../../data/experiments/multiagent_curr_2_fine/imitate/log/args.npz

params loaded from ../../data/experiments/multiagent_curr_2_fine/imitate/log/itr_200.npz
step: 200 / 200
plotting
Saved:  multiagent_rails_col_off_5k_2k_2_fine-multiagent_curr_2_fine_0_0

Don

step: 200 / 200
plotting
Saved:  multiagent_rails_col_off_5k_2k_1_fine-multiagent_curr_1_fine_4_4

Done once.

creating render map for:  multiagent_rails_col_off_5k_2k_2_fine; multiagent_curr_2_fine

running: multiagent_rails_col_off_5k_2k_2_fine

args loaded from ../../data/experiments/multiagent_rails_col_off_5k_2k_2_fine/imitate/log/args.npz

params loaded from ../../data/experiments/multiagent_rails_col_off_5k_2k_2_fine/imitate/log/itr_200.npz
step: 200 / 200
running: multiagent_curr_2_fine

args loaded from ../../data/experiments/multiagent_curr_2_fine/imitate/log/args.npz

params loaded from ../../data/experiments/multiagent_curr_2_fine/imitate/log/itr_200.npz
step: 200 / 200
plotting
Saved:  multiagent_rails_col_off_5k_2k_2_fine-multiagent_curr_2_fine_4_4

Done once.

creating render map for:  multiagent_rails_col_off_5k_2k_3_fine; multiagent_curr_3_fine

running: multiagent_rails_col_off_5k_2k_3_fine

args loaded from ../../data/experiments/multiagent_rails_col_off_5k_2k_3_fine